In [47]:
import json
import numpy as np
import pickle
import utils
import sys
import os
import math
from collections import defaultdict
from collections import Counter

In [4]:
try:
    sys.path.append('/opt/MatterSim/build/')  # local docker or Philly
    import MatterSim
except: 
    # local conda env only
    sys.path.append('/home/hoyeung/Documents/vnla/code/build')  
    import MatterSim

In [5]:
PT_DATA_DIR = "/home/hoyeung/blob_matterport3d/"
os.environ['PT_DATA_DIR'] = "/home/hoyeung/blob_matterport3d/"

EXP_DIR = "/home/hoyeung/blob_experiments/"

In [6]:
with open(PT_DATA_DIR + "asknav/asknav_val_seen.json", "r") as f:
    seen_data = json.load(f)
    
len(seen_data)

4874

In [7]:
with open(PT_DATA_DIR + "asknav/asknav_train.json", "r") as f:
    train_data = json.load(f)
    
len(train_data)

94798

In [8]:
res_dir = "output_philly/20200207_philly_rerun_vnla/dagger_no_ask/20200207_philly_rerun_vnla_nav_sample_ask_teacher_val_seen_for_eval.json"
with open(EXP_DIR + res_dir, "r") as f:
    seen_results = json.load(f)
    
len(seen_results)

4874

In [ ]:
"/home/hoyeung/blob_experiments/output_philly/20200207_philly_rerun_vnla/dagger_no_ask/"

In [9]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_tr_scans.txt', "r") as f:
    asknav_tr_scans = f.read().split('\n')[:-1]
len(asknav_tr_scans)

56

In [10]:
all_panos_to_region = {}
for scan in asknav_tr_scans:
    all_panos_to_region[scan] = utils.load_panos_to_region(scan,"")

In [11]:
viewix_heading_elevation_map = {}
for i in range(0,12):
    viewix_heading_elevation_map[i] = (i*math.pi/6, -math.pi/6)
for i in range(12,24):
    viewix_heading_elevation_map[i] = (i*math.pi/6, 0)
for i in range(24,36):
    viewix_heading_elevation_map[i] = (i*math.pi/6, +math.pi/6)

In [12]:
PT_DATA_DIR = '/home/hoyeung/blob_matterport3d/'
sim = MatterSim.Simulator()
sim.setRenderingEnabled(False)
sim.setDiscretizedViewingAngles(True)
sim.setCameraResolution(640, 480)
sim.setCameraVFOV(math.radians(60))
sim.setNavGraphPath(
    os.path.join(PT_DATA_DIR, 'connectivity'))
sim.init()

### Analysis

In [19]:
increment_30 = 2 * math.pi / 12.0
3.141592653589793/ increment_30

6.0

In [13]:
# Get the most common sequences
# given each starting point image, find the most common gold sequence
# map [(scan, viewpoint, viewix)] : [traj1, traj2, traj3]...

In [21]:
full_trajectories_seen = defaultdict(list)

for dat in seen_data:
    scan = dat['scan']
    start_viewpoint = dat['paths'][0][0]
    viewix = dat['heading'] // 12
    for path in dat['paths']:
        full_trajectories_seen[(scan, start_viewpoint, viewix)].append(path)

len(full_trajectories_seen)

3012

In [23]:
full_trajectories_train = defaultdict(list)

for dat in train_data:
    scan = dat['scan']
    start_viewpoint = dat['paths'][0][0]
    viewix = dat['heading'] // 12
    for path in dat['paths']:
        full_trajectories_train[(scan, start_viewpoint, viewix)].append(path)
        
len(full_trajectories_train)

6774

In [30]:
full_trajectories_seen_res = defaultdict(list)

for res in seen_results:
    scan = res['scan']
    start_viewpoint = res['trajectory'][0][0]
    viewix = res['trajectory'][0][1] // 12
    
    dup_path = [step[0] for step in res['trajectory']]
    step = dup_path[0]
    path = [step]
    for next_step in dup_path:
        if next_step != step:
            step = next_step
            path.append(step)
    
    full_trajectories_seen_res[(scan, start_viewpoint, viewix)].append(path)
    
len(full_trajectories_seen_res)

3012

In [32]:
seen_results_keys = []
for key in full_trajectories_seen_res:
    seen_results_keys.append(key)

### trimmed at step 5

In [74]:
def trim_trajectories(traj_set, length):
    trimmed_traj_set = {}
    for key in traj_set:
        new_paths = [tuple(p[:length]) for p in traj_set[key]]
        trimmed_traj_set[key] = Counter(new_paths).most_common()
    return  trimmed_traj_set

In [56]:
full_trajectories_seen_trimmed_5 = trim_trajectories(full_trajectories_seen, length=5)
full_trajectories_train_trimmed_5 = trim_trajectories(full_trajectories_train, length=5)
full_trajectories_seen_res_trimmed_5 = trim_trajectories(full_trajectories_seen_res, length=5)

In [75]:
full_trajectories_seen_trimmed_8 = trim_trajectories(full_trajectories_seen, length=8)
full_trajectories_train_trimmed_8 = trim_trajectories(full_trajectories_train, length=8)
full_trajectories_seen_res_trimmed_8 = trim_trajectories(full_trajectories_seen_res, length=8)

In [86]:
full_trajectories_seen_trimmed_10 = trim_trajectories(full_trajectories_seen, length=10)
full_trajectories_train_trimmed_10 = trim_trajectories(full_trajectories_train, length=10)
full_trajectories_seen_res_trimmed_10 = trim_trajectories(full_trajectories_seen_res, length=10)

In [79]:
key = seen_results_keys[25]

In [66]:
full_trajectories_seen_trimmed_5[key]

[(('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029',
   '5bc41a6e3b7748149e0e8592c5b4d142',
   'aa4cfd0126dd4c6a9c533ca9cb4a033d'),
  2)]

In [67]:
full_trajectories_seen_res_trimmed_5[key]

[(('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029',
   '5bc41a6e3b7748149e0e8592c5b4d142',
   'aa4cfd0126dd4c6a9c533ca9cb4a033d'),
  1)]

In [68]:
full_trajectories_train_trimmed_5[key]

[(('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029',
   '5bc41a6e3b7748149e0e8592c5b4d142',
   'aa4cfd0126dd4c6a9c533ca9cb4a033d'),
  56),
 (('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029',
   '1a6418a306b8448a8529ffc6cb508d8a'),
  3),
 (('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029',
   '5bc41a6e3b7748149e0e8592c5b4d142'),
  2),
 (('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029',
   '5bc41a6e3b7748149e0e8592c5b4d142',
   'acfc7a38863c412bbde815478879ed54'),
  2),
 (('c2dcba0e507748e0b4eb18690058666e',
   '3e51eeaac8404b31ad8a950bb2bb953d',
   '3f432ddd169d4433979e004d1237d029'),
  1)]

In [80]:
full_trajectories_seen_trimmed_8[key]

[(('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436f528c4bf09e4550079c572f7b',
   '3c6a35e15ada4b649990d6568cce8bd9',
   '397403366d784caf804d741f32fd68b9'),
  1)]

In [81]:
full_trajectories_seen_res_trimmed_8[key]

[(('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436f528c4bf09e4550079c572f7b',
   '3c6a35e15ada4b649990d6568cce8bd9',
   '397403366d784caf804d741f32fd68b9'),
  1)]

In [82]:
full_trajectories_train_trimmed_8[key]

[(('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   'c629c7f1cf6f47a78c45a8ae9ff82247',
   'a7f591c0bf5443a8989e6adf9968cf90',
   'a0a40af004954a90b2b24a0cc6655881'),
  9),
 (('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436f528c4bf09e4550079c572f7b',
   '3c6a35e15ada4b649990d6568cce8bd9',
   '397403366d784caf804d741f32fd68b9'),
  7),
 (('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   'c629c7f1cf6f47a78c45a8ae9ff82247',
   '21fca0d6192940e580587fe317440f56',
   '4b85d61dd3a94e8a812affe78f3a322d'),
  2)]

In [87]:
full_trajectories_seen_trimmed_10[key]

[(('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436f528c4bf09e4550079c572f7b',
   '3c6a35e15ada4b649990d6568cce8bd9',
   '397403366d784caf804d741f32fd68b9',
   'c23f26401359426982d11ca494ee739b',
   '29fb3c58b29348558d36a9f9440a1379'),
  1)]

In [88]:
full_trajectories_seen_res_trimmed_10[key]

[(('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436f528c4bf09e4550079c572f7b',
   '3c6a35e15ada4b649990d6568cce8bd9',
   '397403366d784caf804d741f32fd68b9',
   'c23f26401359426982d11ca494ee739b',
   '29fb3c58b29348558d36a9f9440a1379'),
  1)]

In [89]:
full_trajectories_train_trimmed_10[key]

[(('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   'c629c7f1cf6f47a78c45a8ae9ff82247',
   'a7f591c0bf5443a8989e6adf9968cf90',
   'a0a40af004954a90b2b24a0cc6655881',
   '24e8ebf554d742069e81afd7eb3369c7',
   '4214a9b1607e4cbea8dcdb42a4952796'),
  9),
 (('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436f528c4bf09e4550079c572f7b',
   '3c6a35e15ada4b649990d6568cce8bd9',
   '397403366d784caf804d741f32fd68b9',
   'c23f26401359426982d11ca494ee739b',
   'a9722e0b87a94efcb5c7adbbdb003241'),
  4),
 (('883d1b6540764d2f989271cf0ca56167',
   'a7843f11f41a475792035f544feafaa8',
   '521886391b7849c9b1e458e8e65760de',
   '2fdd7ac33e4e4e5ab38353ba6ab6cc7d',
   '69fad7dd177847dbabf69e8fb7c00ddf',
   '55e4436